In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# שלב 1: איחוד הנתונים
# טוענים כל קובץ ל-DataFrame
df_1= pd.read_csv('Shot Csvs/df_1.csv')
df_2=pd.read_csv('Shot Csvs/df_2.csv')
df_3=pd.read_csv('Shot Csvs/df_3.csv')

# מאחדים את הנתונים לקובץ אחד
df = pd.concat([df_1, df_2,df_3], ignore_index=True) 

# המרת עמודת HitLocalTime לפורמט datetime
try:
    df['HitLocalTime'] = pd.to_datetime(df['HitLocalTime'].str.split(' GMT').str[0], format='%a %b %d %Y %H:%M:%S', errors='coerce')
    print("HitLocalTime after conversion:\n", df['HitLocalTime'].head())
except Exception as e:
    print(f"Error converting HitLocalTime: {e}")

# חישוב הזמן בשניות מתחילת היום
df['TimeInSeconds'] = df['HitLocalTime'].dt.hour * 3600 + df['HitLocalTime'].dt.minute * 60 + df['HitLocalTime'].dt.second

# הצגה של העמודה החדשה
print("TimeInSeconds:\n", df[['HitLocalTime', 'TimeInSeconds']].head())


HitLocalTime after conversion:
 0   2021-07-28 18:06:42
1   2021-07-28 18:06:47
2   2021-07-28 18:06:52
3   2021-07-28 18:07:00
4   2021-07-28 18:07:05
Name: HitLocalTime, dtype: datetime64[ns]
TimeInSeconds:
          HitLocalTime  TimeInSeconds
0 2021-07-28 18:06:42          65202
1 2021-07-28 18:06:47          65207
2 2021-07-28 18:06:52          65212
3 2021-07-28 18:07:00          65220
4 2021-07-28 18:07:05          65225


In [2]:

# חישוב מרחק הזריקה (אוקלידי)
df['ShotDistance'] = np.sqrt((df['HitX'] - df['OriginX'])**2 + (df['HitY'] - df['OriginY'])**2)


# חישוב זווית הזריקה (במעלות) ביחס לציר האופקי
df['ShotAngle'] = np.degrees(np.arctan2(df['HitY'] - df['OriginY'], df['HitX'] - df['OriginX']))

# התאמת זוויות כדי להבטיח שהן בטווח של -90 עד 90 מעלות
df['ShotAngle'] = df['ShotAngle'].apply(lambda angle: angle if -90 <= angle <= 90 else angle - 180 if angle > 90 else angle + 180)

# הצגה של התוצאות
print("Shot Distance and Angle:\n", df[['HitX', 'HitY', 'OriginX', 'OriginY', 'ShotDistance', 'ShotAngle']].head())


Shot Distance and Angle:
     HitX  HitY  OriginX  OriginY  ShotDistance  ShotAngle
0   36.0 -23.0    -2081      249   2134.402258  -7.321462
1  -13.0 -48.0    -2130      217   2133.521502  -7.135009
2 -141.0 -84.0    -1980      204   1861.414785  -8.900617
3   27.0  90.0    -1802       46   1829.529174   1.378091
4   72.0 -37.0    -1995      280   2091.166660  -8.719082


In [3]:

# חישוב מרחק הזריקה (אוקלידי)
df['ShotDistance'] = np.sqrt((df['HitX'] - df['OriginX'])**2 + (df['HitY'] - df['OriginY'])**2)

# חישוב זווית הזריקה (במעלות) ביחס לציר האופקי
df['ShotAngle'] = np.degrees(np.arctan2(df['HitY'] - df['OriginY'], df['HitX'] - df['OriginX']))

# התאמת זוויות כדי להבטיח שהן בטווח של -90 עד 90 מעלות
df['ShotAngle'] = df['ShotAngle'].apply(lambda angle: angle if -90 <= angle <= 90 else angle - 180 if angle > 90 else angle + 180)

# הצגה של התוצאות
print("Shot Distance and Angle:\n", df[['HitX', 'HitY', 'OriginX', 'OriginY', 'ShotDistance', 'ShotAngle']].head())


Shot Distance and Angle:
     HitX  HitY  OriginX  OriginY  ShotDistance  ShotAngle
0   36.0 -23.0    -2081      249   2134.402258  -7.321462
1  -13.0 -48.0    -2130      217   2133.521502  -7.135009
2 -141.0 -84.0    -1980      204   1861.414785  -8.900617
3   27.0  90.0    -1802       46   1829.529174   1.378091
4   72.0 -37.0    -1995      280   2091.166660  -8.719082


In [4]:
# ניקוי נתונים עם מרחק קצר מדי (מתחת ל-3 מטרים)
df = df[df['ShotDistance'] >= 3000]  # מתחת ל-300 ס"מ (3 מטר)

# המרה של עמודת Made ל-1 עבור TRUE ול-0 עבור FALSE
df['Made'] = df['Made'].apply(lambda x: 1 if x else 0)

# המרה של עמודת HitBackboard ל-1 עבור TRUE ול-0 עבור FALSE
df['HitBackboard'] = df['HitBackboard'].apply(lambda x: 1 if x else 0)



# יצירת עמודות בינאריות לעמדות על סמך הזווית
df['Corner_Right'] = np.where((-90 <= df['ShotAngle']) & (df['ShotAngle'] < -60), 1, 0)  # פינה ימין
df['Wing_Right'] = np.where((-60 <= df['ShotAngle']) & (df['ShotAngle'] < -20), 1, 0)  # חצי פינה ימין
df['TOP'] = np.where((-20 <= df['ShotAngle']) & (df['ShotAngle'] <= 20), 1, 0)  # אמצע
df['Wing_Left'] = np.where((20 < df['ShotAngle']) & (df['ShotAngle'] <= 60), 1, 0)  # חצי פינה שמאל
df['Corner_Left'] = np.where((60 < df['ShotAngle']) & (df['ShotAngle'] <= 90), 1, 0)  # פינה שמאל


In [5]:
from sklearn.preprocessing import StandardScaler

# יצירת ClusterID בהתבסס על התנאי שהגדרת:
# כל קלאסטר מתחלף כאשר ההפרש בין הזריקות עולה על 2 דקות
df = df.sort_values(by=['HitLocalTime'])  # מיון לפי הזמן
df['ClusterID'] = 0
current_cluster = 0
previous_time = df['HitLocalTime'].iloc[0]

for idx, row in df.iterrows():
    time_diff = (row['HitLocalTime'] - previous_time).total_seconds() / 60  # ההפרש בדקות
    if time_diff > 1:  # מתחילים קלאסטר חדש אם ההפרש גדול מ-2 דקות
        current_cluster += 1
    df.at[idx, 'ClusterID'] = current_cluster
    previous_time = row['HitLocalTime']


#רשימת עמודות לנרמול
columns_to_normalize = ['ShotDistance', 'ShotAngle', 'TimeInSeconds', 'HitX', 'HitY', 'OriginX', 'OriginY']

# יצירת אובייקט StandardScaler
scaler = StandardScaler()

# ביצוע נרמול של העמודות
normalized_data = scaler.fit_transform(df[columns_to_normalize])

# הוספת העמודות המנורמלות לדאטה פריים עם שמות מציינים נורמליזציה
for i, col in enumerate(columns_to_normalize):
    df[f'Norm_{col}'] = normalized_data[:, i]


# שמירת הקובץ עם העמודות המנורמלות
df.to_csv('Multi_df.csv', index=False)
print("קובץ ה-CSV נשמר בהצלחה עם עמודות מנורמלות.")



קובץ ה-CSV נשמר בהצלחה עם עמודות מנורמלות.
